In [14]:
import numpy as np
import os
import random 
import cv2

IMAGE_SIZE = 50

In [3]:
data = np.load('train.npy',allow_pickle = True)
train_data = data[:-500]
test_data = data[-500:]

In [4]:
print(train_data[0])

[array([[ 55,  52,  51, ...,  24,  18,  44],
       [ 51,  56,  46, ...,  37,  40,  35],
       [ 48,  60,  53, ...,  35,  35,  35],
       ...,
       [ 22,  44,  51, ...,  39,  43,  23],
       [ 24,  23,  23, ..., 134,  15,  14],
       [ 22,  28,  88, ...,  81,  89,  42]], dtype=uint8)
 array([1, 0])]


In [5]:
X = np.array([i[0] for i in train_data]).reshape(-1,50,50,1)
y = [i[1] for i in train_data]

test_x = np.array([i[0] for i in test_data]).reshape(-1,50,50,1)
test_y = [i[1] for i in test_data]

In [6]:
import tensorflow
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

W1105 19:13:56.423406 4553717184 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W1105 19:13:56.424968 4553717184 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W1105 19:13:56.438104 4553717184 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W1105 19:13:56.444534 4553717184 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tflearn/config.py:123: The name tf.get_collection is deprecated.

In [7]:
#CREATING THE MODEL
convnet = input_data(shape=[None, 50, 50, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=1e-3, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

W1105 19:13:56.478313 4553717184 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tflearn/layers/core.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1105 19:13:56.482247 4553717184 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tflearn/initializations.py:119: calling UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1105 19:13:56.484794 4553717184 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/util/deprecation.py:507: UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) is deprecated and will be removed in a future version.
Instructions for u

In [8]:
tensorflow.reset_default_graph()

In [17]:
model.fit({'input': X}, {'targets': y}, n_epoch=20, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=100, show_metric=True, run_id='pXvg_e1')

Training Step: 579  | total loss: 0.15183 | time: 2.271s
| Adam | epoch: 020 | loss: 0.15183 - acc: 0.9743 -- iter: 1792/1802
Training Step: 580  | total loss: 0.13787 | time: 3.363s
| Adam | epoch: 020 | loss: 0.13787 - acc: 0.9769 | val_loss: 0.87219 - val_acc: 0.7760 -- iter: 1802/1802
--


In [18]:
model.save('pXvg_e1.tflearn')

In [9]:
model.predict('/Users/athreya/desktop/GAN_Exploration/PicassoArtClassifier/ProgramFiles/FlaskApp/static/pc124.jpg')

ValueError: could not convert string to float: '/Users/athreya/desktop/GAN_Exploration/PicassoArtClassifier/ProgramFiles/FlaskApp/static/pc124.jpg'

In [10]:
def predict(image):
    t_image = cv2.imread(image)
    try:
        t_image = cv2.resize(t_image,(IMAGE_SIZE,IMAGE_SIZE))
    except:
        print("broken")
    t_image = np.array(t_image).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1)
    prediction = model.predict(t_image)
    if prediction[2][0]>0.5:
        print('picasso')
        print(prediction[2][0])
    else:
        print('van gogh')
        print(prediction[2][1])

In [16]:
predict('/Users/athreya/desktop/GAN_Exploration/PicassoArtClassifier/ProgramFiles/FlaskApp/static/picasso_test.jpg')

van gogh
0.5127544
